In [46]:
import pandas as pd
import pathlib
import plotly.express as px

## Calificar examen estudiantes: Dimensión 7

In [47]:
ruta_git = pathlib.Path().cwd().parent.parent.parent

In [48]:
est = pd.read_excel(ruta_git/'data/descargables/EstudiantesCFK.xlsx')

print(est.shape)
est.columns

(16469, 74)


Index(['N registro', 'Deseo participar en el estudio', 'Código IE', 'Grupo',
       'Nombre', 'Fecha', 'ID', 'Número de lista', 'Edad', 'Sexo',
       'Sector vivienda', 'Internet', 'Uso del dispositivo móvil',
       'Nivel escolaridad madre', 'Nivel escolaridad padre', 'Ocupación madre',
       'Ocupación padre', '¿Con quién vives?', 'Grado', '1.1. Ingeniería',
       '1.2 Matemáticas', '1.3 Educación', '1.4 Medicina',
       '1.5 Diseño gráfico', '1.6 Química', '1.7 Enfermería',
       '1.8 Desarrollo de software',
       '2.1 Soy capaz de sacar buenas notas en esta asignatura',
       '2.2 Si me va bien en esta asignatura, me ayudará en mi futura ocupación',
       '2.3 A mis padres les gustaría que eligiera un futuro profesional relacionado a esta asignatura',
       '2.4 Sé de alguien en mi familia que utiliza conocimientos relacionados a esta asignatura en su ocupación',
       'Comentarios 1-2', 'Un algoritmo es:',
       '¿Para qué sirven los algoritmos?', 'Un bucle es:',
    

##### Pasar respuestas a números

In [49]:
respuestas_correctas = {
    "Un algoritmo es:":"Una secuencia lógica de pasos para realizar una tarea",
    "¿Para qué sirven los algoritmos?":"Para planificar la solución a un problema",
    "Un bucle es:":"Un conjunto de instrucciones que se ejecuta mientras se cumpla una condición",
    "¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?":"a.",
    "¿Qué mensaje deseaba enviar la líder Wayuú?":"c. Nublado",
    "¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?":"a.",
    "¿Cuál será la foto con más vistas?":"c) Julio",
    "Ayuda al robot verde a salir del laberinto":"b.",
    "Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?":"c) Si Óscar empaca Deditos para merendar, puede hacer Arroz de pollo para almorzar",
    "¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?":"a.",
    "¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?":"a) La botella B debe ser verde"}

de_acuerdo = {"Totalmente en desacuerdo":1,"En desacuerdo":2, "Neutro":3,
"De acuerdo":4,"Totalmente de acuerdo":5}

interesa_carreras = {"No la conozco":0,"La evitaría":1,"Me interesa poco":2,"Está entre mis preferidas":3}

estereotipo_h = {'a) Seguramente un hombre':2, 'b) Quizás un hombre':1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': -1, 'e) Seguramente una mujer':-2}

estereotipo_m = {'a) Seguramente un hombre':-2, 'b) Quizás un hombre':-1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': 1, 'e) Seguramente una mujer':2}
#The scores range from -2= ‘counter- stereotypic answer’ to 2= ‘stereotype congruent answer’ for all items.
genero_estereotipos = {'5.1 ¿Quién crees que ganará el concurso de matemáticas?':estereotipo_h,
                       '5.2 ¿Quién crees que es capitán del barco?':estereotipo_h,
                       '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?':estereotipo_m,
                       '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?':estereotipo_m,
                       '5.5 ¿Quién es la persona que trabaja en educación?':estereotipo_m,
                       '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?':estereotipo_h}


In [50]:
col_index = ['Edad', 'Sexo', 'Código IE', '¿Te reconoces como una persona con algún tipo de discapacidad?']
col_caract = ['N registro','Edad', 'Sexo', 'Sector vivienda', 'Internet', 'Uso del dispositivo móvil',
              'Nivel escolaridad madre', 'Nivel escolaridad padre', 'Ocupación madre',
              'Ocupación padre', '¿Con quién vives?', 'Grado', 'Código IE',
              'Grupo', 'Conoce GreenTIC', 'Número de lista', '¿Te reconoces como una persona con algún tipo de discapacidad?']
#col_conocePC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[0]
#col_PC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[1:5]
col_autoeficacia = est.filter(regex=r'^3.*', axis=1).columns.tolist()
col_carreras = est.filter(regex=r'^1.', axis=1).columns.tolist()
col_interes = est.filter(regex=r'^2.1|2.2', axis=1).columns.tolist()
col_conocimiento = list(respuestas_correctas.keys())
col_ambiental = est.filter(regex=r'^4.', axis=1).columns.tolist()
col_genero = est.filter(regex=r'^5.', axis=1).columns.tolist()
col_genero

['5.1 ¿Quién crees que ganará el concurso de matemáticas?',
 '5.2 ¿Quién crees que es capitán del barco?',
 '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?',
 '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?',
 '5.5 ¿Quién es la persona que trabaja en educación?',
 '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?']

In [51]:
df_conocimientos = est[['N registro']+col_conocimiento].melt(id_vars='N registro', value_name='Respuesta estudiante', var_name='Pregunta')
df_conocimientos['Respuesta correcta'] = df_conocimientos['Pregunta'].replace(respuestas_correctas)
df_conocimientos['Puntaje conocimiento'] = 1*(df_conocimientos['Respuesta estudiante'] == df_conocimientos['Respuesta correcta'])
df_estudiantes = df_conocimientos.pivot_table(index='N registro', columns='Pregunta', values='Puntaje conocimiento').reset_index()



df_estudiantes= pd.merge(est[col_caract], df_estudiantes, on='N registro')
df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,Un algoritmo es:,Un bucle es:,"¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?",¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?,¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?,¿Cuál será la foto con más vistas?,¿Para qué sirven los algoritmos?,¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?,¿Qué mensaje deseaba enviar la líder Wayuú?,Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,0,0,1,1,1,0,0,1,0,1
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,0,1,0,0,1,0,0,0,0,0
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,0,1,1,0,0,1,0,0,0
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,1,0,0,0,0,0,0,0,1,0
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,1,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16464,18656,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,No sé,Trabaja,No sé,...,0,0,1,1,0,0,0,0,0,1
16465,18658,13.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Maestría,Maestría,Trabaja,Trabaja,...,0,0,1,1,0,1,1,0,1,0
16466,18659,11.0,Femenino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Cuida el hogar,Trabaja,...,0,0,0,0,0,0,0,0,0,0
16467,18660,11.0,Femenino,Urbana,Sí,No tengo acceso a ningún dispositivo electrónico,Secundaría,Secundaría,Trabaja,Trabaja,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# carreras_df = est[['N registro'] + col_carreras].replace(interesa_carreras)
# df_estudiantes = pd.merge(df_estudiantes, carreras_df, on='N registro')
#
# genero_df = est[['N registro']+col_genero].replace(genero_estereotipos)
# df_estudiantes = pd.merge(df_estudiantes, genero_df, on='N registro')

df_ambiente = est[['N registro']+col_ambiental].replace(de_acuerdo)

df_estudiantes = pd.merge(df_estudiantes, df_ambiente, on='N registro')

df_estudiantes = pd.merge(df_estudiantes, est[['N registro']+col_autoeficacia].astype(int), on='N registro')

df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional,3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional,3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional,3.4 Siento que puedo explicar la forma en que las sub-habilidades del pensamiento computacional se correlacionan con la programación,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,1,2,2,3,3,2,1,2,3
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,4,5,5,5,5,5,5,5,5
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,2,1,1,1,3,2,2,2
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,2,3,2,3,4,4,4,1,3,5
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,3,3,3,3,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16464,18656,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,No sé,Trabaja,No sé,...,2,2,1,1,1,3,1,1,3,4
16465,18658,13.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Maestría,Maestría,Trabaja,Trabaja,...,4,2,2,2,3,3,2,4,5,5
16466,18659,11.0,Femenino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Cuida el hogar,Trabaja,...,1,1,1,1,1,1,1,1,1,1
16467,18660,11.0,Femenino,Urbana,Sí,No tengo acceso a ningún dispositivo electrónico,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,1,1,1,1,1,1,1,1


#### Calcular escalas

In [53]:
escalas_dict = {'medioambiente':
                    {'cols':list(est.filter(regex='^4.*').columns),
                     'cargas':[0.620,0.648,0.732,0.638,0.705,0.707,0.670]},
                'autoeficaciaPC':
                    {'cols':list(est.filter(regex='^3.1 |3.[2-5]|^3.7').columns),
                'cargas':[0.724,0.822,0.782,0.745,0.574,0.390]},
                'autoeficaciaProg':{
                    'cols':list(est.filter(regex='^3.6 |3.[8-9]|3.10').columns),
                    'cargas':[0.637,0.490,0.755,0.753]
                }}

In [54]:
for k in list(escalas_dict.keys()):
    df_estudiantes[k] = 100*df_estudiantes[escalas_dict[k]['cols']].add(-1).multiply(escalas_dict[k]['cargas']).sum(axis=1)/(4*sum(escalas_dict[k]['cargas']))

In [55]:
resultados = df_estudiantes[col_conocimiento].sum(axis=1)
media = resultados.mean()
desv = resultados.std()

df_estudiantes['conocimiento'] = 50+(10*(resultados - media)/desv)



In [56]:
df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples,medioambiente,autoeficaciaPC,autoeficaciaProg,conocimiento
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,3,2,1,2,3,57.621822,27.947733,33.538899,57.964655
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,5,5,5,5,5,75.000000,94.909586,100.000000,43.645839
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,3,2,2,2,79.713983,9.673025,18.956357,53.191716
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,4,4,4,1,3,5,92.521186,46.643547,61.034156,43.645839
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,4,3,3,3,71.461864,52.415160,50.000000,53.191716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16464,18656,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,No sé,Trabaja,No sé,...,1,3,1,1,3,4,82.319915,9.573941,47.846300,48.418777
16465,18658,13.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Maestría,Maestría,Trabaja,Trabaja,...,3,3,2,4,5,5,100.000000,37.521675,83.263757,57.964655
16466,18659,11.0,Femenino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Cuida el hogar,Trabaja,...,1,1,1,1,1,1,50.000000,0.000000,0.000000,34.099961
16467,18660,11.0,Femenino,Urbana,Sí,No tengo acceso a ningún dispositivo electrónico,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,1,1,1,1,75.000000,0.000000,0.000000,34.099961


In [57]:
estudiantes = pd.merge(est, df_estudiantes[['N registro', 'conocimiento', 'autoeficaciaPC', 'autoeficaciaProg', 'medioambiente']])

In [60]:
df_interes = est[['N registro']+col_interes].melt(id_vars='N registro', value_name='Seleccion', var_name='Pregunta')
df_interes['Interes en tecnologia'] = 1*(df_interes['Seleccion'].str.contains('Tecnología'))
df_puntaje_interes = df_interes.pivot_table(index='N registro', values=['Interes en tecnologia'], aggfunc=['sum', 'max']).reset_index()
df_puntaje_interes.columns = ['N registro', 'Puntaje interes', 'Interesado en tecnologia']
df_puntaje_interes['Puntaje interes'] = 100*(df_puntaje_interes['Puntaje interes']/2)
#df_puntaje_interes = (100*df_interes.groupby('index')['Interes en tecnologia'].sum()/4).reset_index()
df_puntaje_interes

,N registro,Puntaje interes,Interesado en tecnologia
0,1,100.0,1
1,2,50.0,1
2,3,50.0,1
3,4,100.0,1
4,5,0.0,0
...,...,...,...
16464,18656,0.0,0
16465,18658,50.0,1
16466,18659,0.0,0
16467,18660,100.0,1


In [63]:
df_7 = pd.merge(estudiantes, df_puntaje_interes, on='N registro')
df_7

,N registro,Deseo participar en el estudio,Código IE,Grupo,Nombre,Fecha,ID,Número de lista,Edad,Sexo,...,Tipo de discapacidad,¿Te reconoces como una persona con algún tipo de discapacidad?,Conoce GreenTIC,Instrumento,conocimiento,autoeficaciaPC,autoeficaciaProg,medioambiente,Puntaje interes,Interesado en tecnologia
0,1,Sí,166,2,NaN,19/04,166090203,3,14.0,Masculino,...,NaN,No,No,Encuesta estudiantes,57.964655,27.947733,33.538899,57.621822,100.0,1
1,2,Sí,166,2,NaN,19/04,166090207,7,16.0,Masculino,...,NaN,No,No,Encuesta estudiantes,43.645839,94.909586,100.000000,75.000000,50.0,1
2,3,Sí,166,2,NaN,19/04,166090230,30,14.0,Femenino,...,NaN,No,No,Encuesta estudiantes,53.191716,9.673025,18.956357,79.713983,50.0,1
3,4,Sí,166,2,NaN,19/04,166090206,6,15.0,Masculino,...,NaN,No,No,Encuesta estudiantes,43.645839,46.643547,61.034156,92.521186,100.0,1
4,5,Sí,166,2,NaN,19/04,166090210,10,14.0,Femenino,...,NaN,No,No,Encuesta estudiantes,53.191716,52.415160,50.000000,71.461864,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16464,18656,Sí,208,4,Juanita Ramírez Saavedra,26/05,208090424,24,14.0,Femenino,...,NaN,No,No,Encuesta estudiantes,48.418777,9.573941,47.846300,82.319915,0.0,0
16465,18658,Sí,208,4,Aura Alicia Mendoza Salinas,26/05,208090415,15,13.0,Femenino,...,NaN,No,No,Encuesta estudiantes,57.964655,37.521675,83.263757,100.000000,50.0,1
16466,18659,Sí,113,4,Yarley Andrea Araujo Florez,26/05,113060401,1,11.0,Femenino,...,NaN,No,No,Encuesta estudiantes,34.099961,0.000000,0.000000,50.000000,0.0,0
16467,18660,Sí,113,4,Yuliana Ayala Pacheco,26/05,113060403,3,11.0,Femenino,...,NaN,No,No,Encuesta estudiantes,34.099961,0.000000,0.000000,75.000000,100.0,1


In [66]:
df_7['autoeficaciaProg'].mean()

62.04266068052876

In [67]:
df_7['Media autoeficaciaProg'] = df_7['autoeficaciaProg'].mean()
df_7['Media autoeficaciaPC'] = df_7['autoeficaciaPC'].mean()

In [21]:
df_7['Media nacional interés'] = df_7['Puntaje interes'].mean()


### Calificar Dimensión

In [23]:
cols_dimension7 = ['Puntaje conocimiento', 'Media nacional resultados','Puntaje interes', 'Interesado en tecnologia', 'Conoce PC', 'Puntaje conocimientos PC', 'Puntaje autoeficacia', 'Media nacional autoeficacia', 'Media nacional interés', 'index']
agrupadores_dimension7 = ['mean', 'max', 'mean', 'sum', 'sum', 'mean','mean', 'max', 'max', 'count']
agg_funcs = dict(zip(cols_dimension7, agrupadores_dimension7))

In [24]:
df_institucion = df_7.pivot_table(index=['Código IE'], values=cols_dimension7, aggfunc=agg_funcs).rename(columns={'index':'Cantidad estudiantes'})
df_institucion['% Conoce PC'] = (df_institucion['Conoce PC']/df_institucion['Cantidad estudiantes'])*100
df_institucion['% Interesado'] = (df_institucion['Interesado en tecnologia']/df_institucion['Cantidad estudiantes'])*100

In [25]:
df_institucion['Diferencia resultados'] = (df_institucion['Puntaje conocimiento'] - df_institucion['Media nacional resultados'])*100/df_institucion['Media nacional resultados']

df_institucion['Diferencia Autoeficacia'] = (df_institucion['Puntaje autoeficacia'] - df_institucion['Media nacional autoeficacia'])*100/df_institucion['Media nacional autoeficacia']

df_institucion['Diferencia interes'] = (df_institucion['Puntaje interes'] - df_institucion['Media nacional interés'])*100/df_institucion['Media nacional interés']

In [26]:
df_institucion

,Conoce PC,Interesado en tecnologia,Media nacional autoeficacia,Media nacional interés,Media nacional resultados,Puntaje autoeficacia,Puntaje conocimiento,Puntaje conocimientos PC,Puntaje interes,Cantidad estudiantes,% Conoce PC,% Interesado,Diferencia resultados,Diferencia Autoeficacia,Diferencia interes
Código IE,,,,,,,,,,,,,,,
1.0,90,109,66.752077,40.633652,32.216324,69.466667,28.363636,58.566667,36.333333,150,60.000000,72.666667,-11.958805,4.066675,-10.583146
2.0,35,53,66.752077,40.633652,32.216324,59.104478,24.830393,51.492537,47.388060,67,52.238806,79.104478,-22.926049,-11.456721,16.622694
3.0,41,43,66.752077,40.633652,32.216324,69.904762,29.870130,59.761905,39.285714,63,65.079365,68.253968,-7.282625,4.722977,-3.317294
4.0,24,35,66.752077,40.633652,32.216324,66.222222,33.737374,59.555556,46.666667,45,53.333333,77.777778,4.721364,-0.793765,14.847335
5.0,69,99,66.752077,40.633652,32.216324,73.981651,30.942452,61.422018,55.504587,109,63.302752,90.825688,-3.954119,10.830487,36.597584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250.0,37,50,66.752077,40.633652,32.216324,62.024096,23.767798,54.337349,24.397590,83,44.578313,60.240964,-26.224362,-7.082896,-39.957181
251.0,81,77,66.752077,40.633652,32.216324,66.909091,25.770098,59.504132,27.479339,121,66.942149,63.636364,-20.009192,0.235220,-32.372953
252.0,37,42,66.752077,40.633652,32.216324,62.714286,24.188312,57.946429,33.035714,56,66.071429,75.000000,-24.919082,-6.048937,-18.698634


In [27]:
df_institucion['Nivel ConocePC'] =  pd.cut(df_institucion['% Conoce PC'], bins=[0, 20, 40, 50, 60, 100], labels=['1A', '1B', '2A', '2B', '3'], include_lowest=True)

df_institucion['Nivel PuntajePC'] =  pd.cut(df_institucion['Puntaje conocimientos PC'], bins=[0, 50, 60,70, 100], labels=['1A', '1B',"2A",'3'], include_lowest=True)

df_institucion['Nivel Resultados'] = pd.cut(df_institucion['Diferencia resultados'], bins=[-100, -50, -30,  0, 10,20, 100], labels=['1A', '1B', "2A", "2B", '3', '4'], include_lowest=True)

df_institucion['Nivel Interes'] = pd.cut(df_institucion['% Interesado'], bins=[0, 20, 40, 60, 80, 100], labels=['1A', '1B', "2A", "2B", '4'], include_lowest=True)

df_institucion['Nivel Interes pares'] = pd.cut(df_institucion['Diferencia interes'], bins=[-100,-50, -30, 0 , 200], labels=['1B', "2A", "2B", '4'], include_lowest=True)

df_institucion['Nivel Autoeficacia'] = pd.cut(df_institucion['Puntaje autoeficacia'], bins=[0, 30, 60, 100], labels=['1B', "2A", '4'], include_lowest=True)

df_institucion['Nivel Autoeficacia pares'] = pd.cut(df_institucion['Diferencia Autoeficacia'], bins=[-100, 0, 10, 20, 100], labels=['2A',"2B", "3", '4'], include_lowest=True)


df_institucion

,Conoce PC,Interesado en tecnologia,Media nacional autoeficacia,Media nacional interés,Media nacional resultados,Puntaje autoeficacia,Puntaje conocimiento,Puntaje conocimientos PC,Puntaje interes,Cantidad estudiantes,...,Diferencia resultados,Diferencia Autoeficacia,Diferencia interes,Nivel ConocePC,Nivel PuntajePC,Nivel Resultados,Nivel Interes,Nivel Interes pares,Nivel Autoeficacia,Nivel Autoeficacia pares
Código IE,,,,,,,,,,,,,,,,,,,,,
1.0,90,109,66.752077,40.633652,32.216324,69.466667,28.363636,58.566667,36.333333,150,...,-11.958805,4.066675,-10.583146,2B,1B,2A,2B,2B,4,2B
2.0,35,53,66.752077,40.633652,32.216324,59.104478,24.830393,51.492537,47.388060,67,...,-22.926049,-11.456721,16.622694,2B,1B,2A,2B,4,2A,2A
3.0,41,43,66.752077,40.633652,32.216324,69.904762,29.870130,59.761905,39.285714,63,...,-7.282625,4.722977,-3.317294,3,1B,2A,2B,2B,4,2B
4.0,24,35,66.752077,40.633652,32.216324,66.222222,33.737374,59.555556,46.666667,45,...,4.721364,-0.793765,14.847335,2B,1B,2B,2B,4,4,2A
5.0,69,99,66.752077,40.633652,32.216324,73.981651,30.942452,61.422018,55.504587,109,...,-3.954119,10.830487,36.597584,3,2A,2A,4,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250.0,37,50,66.752077,40.633652,32.216324,62.024096,23.767798,54.337349,24.397590,83,...,-26.224362,-7.082896,-39.957181,2A,1B,2A,2B,2A,4,2A
251.0,81,77,66.752077,40.633652,32.216324,66.909091,25.770098,59.504132,27.479339,121,...,-20.009192,0.235220,-32.372953,3,1B,2A,2B,2A,4,2B
252.0,37,42,66.752077,40.633652,32.216324,62.714286,24.188312,57.946429,33.035714,56,...,-24.919082,-6.048937,-18.698634,3,1B,2A,2B,2B,4,2A


In [29]:
df_institucion['Nivel ConocePC'].value_counts()

3     78
2B    37
2A    11
1A     6
1B     3
Name: Nivel ConocePC, dtype: int64

In [28]:
df_institucion['Dimensión 7'] = df_institucion.filter(regex='^Nivel').min(axis=1)

In [30]:
df_institucion.to_excel('Dimensión 7.xlsx')

In [29]:
df_institucion['Dimensión 7']

Código IE
1.0      1B
2.0      1B
3.0      1B
4.0      1B
5.0      2A
         ..
250.0    1B
251.0    1B
252.0    1B
259.0    1A
297.0    2A
Name: Dimensión 7, Length: 135, dtype: object